# PCA section

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_train_selected_lasso)

In [ ]:
explained_var_lis = np.cumsum(pca.explained_variance_ratio_)
dim_lis = np.array(range(76)) + 1

In [ ]:
elbow_idx= np.argmax(explained_var_lis >= 0.95) + 1
print("elbow index: ", elbow_idx)
elbow_value = explained_var_lis[elbow_idx - 1]

In [ ]:
fig, ax = plt.subplots()
ax.plot(dim_lis,explained_var_lis, linewidth=3, color = "maroon")
ax.set_ylabel("Explained variance")
ax.set_xlabel("Dimensions")

plt.scatter(elbow_idx, elbow_value, color = "green", s =100)
ax.plot([elbow_idx, elbow_idx], [0, elbow_value], linestyle = '--', color = 'grey')
ax.plot([0,elbow_idx], [elbow_value, elbow_value], linestyle = '--', color = "grey")
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)
plt.show()

Therefore we reduced original data dimension to 189 by applying PCA

In [ ]:
X_train_pca = PCA(n_components= 38).fit_transform(X_train_selected_lasso)

In [ ]:
X_train_pca.shape

# Forward step-wise section

However, after visualizing both the cross-validation and training errors against the number of features, we observed that the model's cross-validation error started to gradually decrease around 170 features and began to increase around 240 features, indicating the sign of overfitting

Based on the zoom-in plot, we observed that the cross-validation error decreased slowly, indicating that increasing the number of features provided little improvement to model performance. So we decideed to set the number of selected forward stepwise features to 170

In [ ]:
fig,ax = plt.subplots()
ax.plot(feature_num_lis, training_error_lis, label= "training error", color = "navy")
ax.plot(feature_num_lis, cv_error_lis, label = "cross validation error", color = "magenta")
ax.set(xlim=(120,190))
ax.set_ylabel("Error")
ax.set_xlabel("Number of features")
ax.legend()
plt.show()

# Clustering section

In [ ]:
def build_cluster_oof_features(X_stats, y, k=32, n_splits=5, task="classification"):
    n = len(y)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    oof_dist  = np.zeros((n, k)) # distance to core of each cluster, each song
    oof_mean  = np.zeros((n, 1)) # mean, each song
    oof_size  = np.zeros((n, 1)) # 
    oof_label = np.zeros(n, dtype=int) # cluster labrl, each song

    # 为了在测试时使用：训练完后再用全量拟合一遍
    #scaler_full = StandardScaler().fit(X_stats)
    #Xz_full = scaler_full.transform(X_stats)
    #km_full = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=2048, n_init="auto").fit(Xz_full)

    for tr, va in kf.split(X_stats): # training, validation sets
        # 1) 该折的标准化 + 聚类只在训练折上拟合
        scaler = StandardScaler().fit(X_stats[tr])
        Xz_tr = scaler.transform(X_stats[tr])
        Xz_va = scaler.transform(X_stats[va])

        km = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=2048, n_init="auto").fit(Xz_tr)

        # 2) 给验证折打特征
        oof_dist[va]  = km.transform(Xz_va)        # 到质心距离（越小越近）
        oof_label[va] = km.predict(Xz_va)

        # 3) 只用训练折按簇统计目标
        #lab_tr = km.predict(Xz_tr)
        #mean_by_c = {}  # 每一个cluster 的 mean
        #size_by_c = {} # 每一个cluster 的 size
        #base = y[tr].mean()
        #for c in range(k):
        #    mask = (lab_tr == c) 
        #    size = int(mask.sum())
        #    size_by_c[c] = size #每一个 label 的 size, training set
        #    if size > 0:
        #        m = y[tr][mask].mean() if task == "regression" else y[tr][mask].mean()
        #    else:
        #        m = base
        #    mean_by_c[c] = m

        # 4) 把簇统计赋给验证折样本
        #oof_mean[va, 0] = np.array([mean_by_c[c] for c in oof_label[va]])
        #oof_size[va, 0] = np.array([size_by_c[c] for c in oof_label[va]])

    # 供测试阶段：用全训练集的簇统计
    #lab_full = km_full.predict(Xz_full)
    #full_mean = {c: (y[lab_full==c].mean() if (lab_full==c).sum()>0 else y.mean()) for c in range(k)}
    #full_size = {c: (lab_full==c).sum() for c in range(k)}

    #cluster_stats_full = {"mean": full_mean, "size": full_size}

    return (oof_dist, #oof_mean, oof_size, 
            oof_label,
            #scaler_full, km_full, cluster_stats_full
            )

# acoustic feature section

In [ ]:
combined_matrix_bars = np.concatenate((array_cluster_bars[0], array_cluster_bars[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_bars.shape

combined_matrix_beats = np.concatenate((array_cluster_beats[0], array_cluster_beats[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_beats.shape

combined_matrix_sections = np.concatenate((array_cluster_sections[0], array_cluster_sections[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_sections.shape

combined_matrix_loudness = np.concatenate((array_cluster_loudness[0], array_cluster_loudness[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_loudness.shape

combined_matrix_loudness_time = np.concatenate((array_cluster_loudness_time[0], array_cluster_loudness_time[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_loudness_time.shape

combined_matrix_loudness_start = np.concatenate((array_cluster_loudness_start[0], array_cluster_loudness_start[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_loudness_start.shape

combined_matrix_segments_start = np.concatenate((array_cluster_segments_start[0], array_cluster_segments_start[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_segments_start.shape

combined_matrix_tatums_start = np.concatenate((array_cluster_tatums_start[0], array_cluster_tatums_start[1].reshape(-1,1)), axis=1) #final matrix but not standardized yet
combined_matrix_tatums_start.shape

In [ ]:
combined_matrix_bars.shape, combined_matrix_beats.shape, combined_matrix_sections.shape,combined_matrix_loudness.shape, combined_matrix_loudness_time.shape

In [ ]:
all_matrix_combiend = np.concatenate([combined_matrix_bars, combined_matrix_beats, combined_matrix_sections, combined_matrix_loudness, 
                                      combined_matrix_loudness_time
                                      ], axis=1)

In [ ]:
np.save("acoustic_features_part.npy", all_matrix_combiend)

In [ ]:
scaler = StandardScaler()
all_matrix_combiend = scaler.fit_transform(all_matrix_combiend)

np.save("acoustic_features_standardized_part.npy", all_matrix_combiend)

In [ ]:
cluster_distance_bars = array_cluster_bars[0]
cluster_distance_beats = array_cluster_beats[0]
cluster_distance_loudness = array_cluster_loudness[0]
cluster_distance_loudness_time = array_cluster_loudness_time[0]

np.save("cluster_distance_bars", cluster_distance_bars)
np.save("cluster_distance_beats", cluster_distance_beats)
np.save("cluster_distance_loudness", cluster_distance_loudness)
np.save("cluster_distance_loudness_time", cluster_distance_loudness_time)